# Installing Required packages

## Lets connect to the database 

In [8]:
import os
import sys
import pandas as pd

# Manual parsing of .env file
with open('.env') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            key, value = line.strip().split('=', 1)
            os.environ[key] = value

# Access the environment variables
db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")
db_port = os.getenv("DB_PORT")

# Print connection details (with masked password)
print(f"Connecting to MySQL at {db_host}:{db_port}")
print(f"Database: {db_name}")
print(f"User: {db_user}")
print(f"Password: {'*' * len(db_password) if db_password else 'None'}")

# Import SQLAlchemy after installing it
from sqlalchemy import create_engine

# Create MySQL connection string
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

try:
    # Try to create the engine
    print("Creating SQLAlchemy engine...")
    engine = create_engine(connection_string, echo=False)
    
    # Try to connect
    print("Attempting to connect...")
    connection = engine.connect()
    print("Connection successful!")
    connection.close()
    
except ImportError as e:
    print(f"Driver error: {e}")
    print("Please install the required database driver:")
    print("pip install pymysql")
    
except Exception as e:
    print(f"Error connecting to the database: {e}")
    print(f"Python version: {sys.version}")

ModuleNotFoundError: No module named 'pandas'

In [11]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

# Employee Demographics & Diversity (Basic & Aggregation Queries)

## What is the total number of employees?

In [10]:
import pandas as pd

query = """SELECT COUNT(*) AS total_employees
FROM employees;
"""

total_employees = pd.read_sql(query, engine)
print(total_employees)

ModuleNotFoundError: No module named 'pandas'

## What is the gender distribution of employees?

In [ ]:
query = """SELECT 
    gender,
    COUNT(*) AS employee_count,
    COUNT(*) * 1.0 / (SELECT COUNT(*) FROM employees) AS gender_ratio
FROM employees
GROUP BY gender;"""
gender_distribution = pd.read_sql(query,engine)

gender_distribution


Basic grouping with a subquery for proportional analysis.

## What is the race/ethnicity distribution across different departments?

In [ ]:
query = """SELECT 
    department,
    race,
    COUNT(*) AS race_count,
    COUNT(*) * 1.0 / SUM(COUNT(*)) OVER (PARTITION BY department) AS race_ratio
FROM employees
GROUP BY department, race
ORDER BY department, race_count DESC;"""

race_distribution = pd.read_sql(query,engine)
race_distribution

Uses window functions for departmental race proportions.

## What is the age distribution of employees?

In [ ]:
query ="""SELECT 
    FLOOR(DATEDIFF(YEAR, birthdate, '2025-03-12') / 10) * 10 AS age_range_start,
    COUNT(*) AS employee_count
FROM employees
GROUP BY FLOOR(DATEDIFF(YEAR, birthdate, '2025-03-12') / 10) * 10
ORDER BY age_range_start;"""

age_distribution =pd.read_sql(query,engine)
age_distribution



groups ages into decades using date functions and aggregation

## What is the average age of employees in each department?

In [ ]:
query ="""SELECT 
    department,
    AVG(DATEDIFF(YEAR, birthdate, '2025-03-12')) AS avg_age,
    MIN(DATEDIFF(YEAR, birthdate, '2025-03-12')) AS min_age,
    MAX(DATEDIFF(YEAR, birthdate, '2025-03-12')) AS max_age
FROM employees
GROUP BY department
ORDER BY avg_age DESC;"""
avg_age_department= pd.read_sql(query,engine)
avg_age_department

Combines multiple aggregations for a comprehensive departmental view.

# Tenure & Employee Retention (Date Functions & Aggregation)

## What is the average tenure of employees in each department?

In [ ]:
query = """SELECT 
    department,
    AVG(DATEDIFF(YEAR, hire_date, COALESCE(termdate, '2025-03-12'))) AS avg_tenure_years
FROM employees
GROUP BY department
ORDER BY avg_tenure_years DESC;"""
avg_tenure_dept = pd.read_sql(query,engine)
avg_tenure_dept

Uses COALESCE to handle active employees and date differences.

## Who are the top 5 longest-serving employees?

In [ ]:
query = """SELECT TOP 5
    id,
    first_name,
    last_name,
    hire_date,
    DATEDIFF(YEAR, hire_date, COALESCE(termdate, '2025-03-12')) AS tenure_years
FROM employees
ORDER BY tenure_years DESC;"""
top_5_longtest_serving = pd.read_sql(query,engine)

top_5_longtest_serving

Simple ranking with TOP for seniority

## What percentage of employees have left the company, and what is the average tenure of terminated employees?

In [ ]:
query = """SELECT 
    COUNT(CASE WHEN termdate IS NOT NULL THEN 1 END) * 1.0 / COUNT(*) AS termination_rate,
    AVG(CASE WHEN termdate IS NOT NULL THEN DATEDIFF(YEAR, hire_date, termdate) END) AS avg_tenure_terminated
FROM employees;"""
term_rate  = pd.read_sql(query,engine)
term_rate

Conditional aggregation for turnover metrics.

## What is the attrition rate for each department?

In [ ]:
query = """SELECT 
    department,
    COUNT(CASE WHEN termdate IS NOT NULL THEN 1 END) AS terminations,
    COUNT(*) AS total_employees,
    COUNT(CASE WHEN termdate IS NOT NULL THEN 1 END) * 1.0 / COUNT(*) AS attrition_rate
FROM employees
GROUP BY department
ORDER BY attrition_rate DESC;"""
attrition_rate_perdept = pd.read_sql(query,engine)
attrition_rate_perdept

## How many employees were hired in each year, and what is the hiring trend?

Departmental turnover analysis with ratios.

In [ ]:
query = """SELECT 
    YEAR(hire_date) AS hire_year,
    COUNT(*) AS hires,
    SUM(COUNT(*)) OVER (ORDER BY YEAR(hire_date)) AS cumulative_hires
FROM employees
GROUP BY YEAR(hire_date)
ORDER BY hire_year;"""
hiring_trends=pd.read_sql(query,engine)
hiring_trends

Tracks hiring trends with a cumulative sum using window functions.

# Advanced Queries (CTEs, Window Functions, Rank, Performance)

## Rank employees by tenure within each department.

In [ ]:
query = """SELECT 
    id,
    first_name,
    last_name,
    department,
    DATEDIFF(YEAR, hire_date, COALESCE(termdate, '2025-03-12')) AS tenure_years,
    RANK() OVER (PARTITION BY department ORDER BY DATEDIFF(YEAR, hire_date, COALESCE(termdate, '2025-03-12')) DESC) AS tenure_rank
FROM employees
ORDER BY department, tenure_rank;"""
tenure_rank = pd.read_sql(query,engine)
tenure_rank

Uses RANK() for intra-departmental seniority.

## Identify employees who were hired and left within 2 years.

In [ ]:
query = """SELECT 
    id,
    first_name,
    last_name,
    hire_date,
    termdate,
    DATEDIFF(YEAR, hire_date, termdate) AS tenure_years
FROM employees
WHERE termdate IS NOT NULL 
AND DATEDIFF(YEAR, hire_date, termdate) <= 2
ORDER BY hire_date;"""

left_within2_years = pd.read_sql(query,engine)
left_within2_years

Filters short-tenure terminations with date logic.

## Calculate the rolling 12-month average of employee terminations.

In [ ]:
query = """WITH Terminations AS (
    SELECT 
        DATEADD(MONTH, DATEDIFF(MONTH, 0, termdate), 0) AS term_month,
        COUNT(*) AS term_count
    FROM employees
    WHERE termdate IS NOT NULL
    GROUP BY DATEADD(MONTH, DATEDIFF(MONTH, 0, termdate), 0)
)
SELECT 
    term_month,
    term_count,
    AVG(term_count * 1.0) OVER (ORDER BY term_month ROWS BETWEEN 11 PRECEDING AND CURRENT ROW) AS rolling_12month_avg
FROM Terminations
ORDER BY term_month;"""
avg_12month_terminations= pd.read_sql(query,engine)
avg_12month_terminations

Uses a rolling window with date truncation for trend analysis.

## Find employees whose hire date is within 30 days of their birth month.

In [ ]:
query = """SELECT 
    id,
    first_name,
    last_name,
    birthdate,
    hire_date,
    DATEDIFF(DAY, DATEADD(YEAR, DATEDIFF(YEAR, birthdate, hire_date), birthdate), hire_date) AS days_from_birthday
FROM employees
WHERE ABS(DATEDIFF(DAY, DATEADD(YEAR, DATEDIFF(YEAR, birthdate, hire_date), birthdate), hire_date)) <= 30;"""
hire_date_birth_month = pd.read_sql(query,engine)
hire_date_birth_month

Complex date comparison to align hire dates with birthdays.

## Detect anomalies in termdates (e.g., employees who left before their hire date).

In [ ]:
query ="""SELECT 
    id,
    first_name,
    last_name,
    hire_date,
    termdate
FROM employees
WHERE termdate IS NOT NULL 
AND termdate < hire_date;"""

termdate_anomalies= pd.read_sql(query,engine)
termdate_anomalies

Identifies data quality issues with logical date checks.

# Performance Optimization (Indexes, Partitions, Views)

## Create an indexed view that shows active employees by department

In [ ]:
# First, create the view (execute as a separate statement)
create_view_query = """CREATE VIEW vw_ActiveEmployeesByDept
WITH SCHEMABINDING
AS
    SELECT 
        department,
        COUNT_BIG(*) AS active_employee_count
    FROM dbo.employees
    WHERE termdate IS NULL
    GROUP BY department;"""

# Execute the CREATE VIEW statement
engine.execute(create_view_query)

# Second, create the index (as a separate statement)
create_index_query = """CREATE UNIQUE CLUSTERED INDEX IDX_vw_ActiveEmployeesByDept 
                        ON vw_ActiveEmployeesByDept(department);"""

# Execute the CREATE INDEX statement
engine.execute(create_index_query)

# Now query the view to get the data
query = "SELECT * FROM vw_ActiveEmployeesByDept;"
active_employees_by_dept = pd.read_sql(query, engine)
active_employees_by_dept

Optimizes queries for active employees with an indexed view.

## Optimize a query to find employees hired before 2010 with indexes.

In [ ]:
# First, create the nonclustered index
create_index_query = """CREATE NONCLUSTERED INDEX IDX_Employees_HireDate 
                        ON employees(hire_date);"""

# Execute the CREATE INDEX statement
engine.execute(create_index_query)

# Then, query the data using the index
query = """SELECT 
    id,
    first_name,
    last_name,
    hire_date
FROM employees
WHERE hire_date < '2010-01-01'
ORDER BY hire_date;"""

# Store the result in a DataFrame
early_hires = pd.read_sql(query, engine)

# Display the DataFrame
early_hires

Improves performance with a non-clustered index on `hire_date`.

## Use partitioning to speed up queries for employees by location

In [ ]:
# Step 1: Create the partition function
partition_function_query = """CREATE PARTITION FUNCTION pf_LocationState (NVARCHAR(50))
AS RANGE LEFT FOR VALUES ('Ohio', 'Texas');"""
engine.execute(partition_function_query)

# Step 2: Create the partition scheme
partition_scheme_query = """CREATE PARTITION SCHEME ps_LocationState
AS PARTITION pf_LocationState ALL TO ([PRIMARY]);"""
engine.execute(partition_scheme_query)

# Step 3: Create the partitioned table
create_table_query = """CREATE TABLE employees_partitioned (
    id NVARCHAR(10), first_name NVARCHAR(50), last_name NVARCHAR(50), birthdate DATE, 
    gender NVARCHAR(10), race NVARCHAR(50), department NVARCHAR(50), jobtitle NVARCHAR(50), 
    location NVARCHAR(50), hire_date DATE, termdate DATE, location_city NVARCHAR(50), 
    location_state NVARCHAR(50)
) ON ps_LocationState(location_state);"""
engine.execute(create_table_query)

# Step 4: Insert data from the original table
insert_data_query = """INSERT INTO employees_partitioned SELECT * FROM employees;"""
engine.execute(insert_data_query)

# Step 5: Query data from the partitioned table
query = """SELECT * FROM employees_partitioned WHERE location_state = 'Ohio';"""
ohio_employees = pd.read_sql(query, engine)

# Display the results
ohio_employees

Partitions data by state for faster location-based queries.

## Write a stored procedure to get employee counts by department for a given date range.

In [ ]:
# Step 1: Create the stored procedure
create_procedure_query = """CREATE PROCEDURE sp_EmployeeCountByDept
    @StartDate DATE,
    @EndDate DATE
AS
BEGIN
    SELECT 
        department,
        COUNT(*) AS employee_count
    FROM employees
    WHERE hire_date <= @EndDate 
    AND (termdate IS NULL OR termdate >= @StartDate)
    GROUP BY department
    ORDER BY employee_count DESC;
END;"""

# Execute the CREATE PROCEDURE statement
engine.execute(create_procedure_query)

# Step 2: Execute the stored procedure with parameters
from sqlalchemy import text

# Define the parameters
params = {
    'StartDate': '2020-01-01', 
    'EndDate': '2025-03-12'
}

# Call the stored procedure
exec_query = text("EXEC sp_EmployeeCountByDept @StartDate = :StartDate, @EndDate = :EndDate;")
dept_counts = pd.read_sql(exec_query, engine, params=params)

# Display the results
dept_counts

Encapsulates logic for reusable, parameterized reporting.

## Create a materialized view for high-speed analytics on employee demographics

In [ ]:
# Step 1: Create the view with SCHEMABINDING
create_view_query = """CREATE VIEW vw_EmployeeDemographics
WITH SCHEMABINDING
AS
    SELECT 
        department,
        gender,
        race,
        AVG(DATEDIFF(YEAR, birthdate, '2025-03-12')) AS avg_age,
        COUNT_BIG(*) AS employee_count
    FROM dbo.employees
    GROUP BY department, gender, race;"""

# Execute the CREATE VIEW statement
engine.execute(create_view_query)

# Step 2: Create the unique clustered index on the view
# Note: Remove the GO statement as it's not needed for Python execution
create_index_query = """CREATE UNIQUE CLUSTERED INDEX IDX_vw_EmployeeDemographics 
                        ON vw_EmployeeDemographics(department, gender, race);"""

# Execute the CREATE INDEX statement
engine.execute(create_index_query)

# Optional Step 3: Query the view to see the results
query = "SELECT * FROM vw_EmployeeDemographics ORDER BY department, gender, race;"
employee_demographics = pd.read_sql(query, engine)

# Display the results
employee_demographics

Pre-aggregates demographics for rapid analytics.